<a href="https://colab.research.google.com/github/JaeminKIM-Irene/FC_AI_ML_Project/blob/main/irenekim/ML_Project_LightGBM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 설치에 필요한 라이브러리들이 있다면 모두 적어둡니다. anaconda에 기본적으로 설치되지 않은 라이브러리들을 적어두세요.
!pip install lightgbm optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.2/404.2 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.0/226.0 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 8.0 MB/s eta 0:00:00


In [28]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from lightgbm import LGBMClassifier

from sklearn.model_selection import KFold

import optuna

from sklearn.model_selection import StratifiedKFold
from functools import partial

from sklearn.metrics import roc_auc_score

In [20]:
# google drive mount
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
# read data
result_list = []
base_url = '/content/drive/MyDrive/Colab Notebooks/data/playground-series-s3e23/'
train = pd.read_csv(base_url+'train.csv', index_col='id')
test = pd.read_csv(base_url+'test.csv', index_col='id')
submission = pd.read_csv(base_url+'sample_submission.csv', index_col='id')

with pd.option_context('display.min_rows', 6) :
  display(train)

,loc,v(g),ev(g),iv(g),n,v,l,d,i,e,...,lOCode,lOComment,lOBlank,locCodeAndComment,uniq_Op,uniq_Opnd,total_Op,total_Opnd,branchCount,defects
id,,,,,,,,,,,,,,,,,,,,,
0,22.0,3.0,1.0,2.0,60.0,278.63,0.06,19.56,14.25,5448.79,...,17,1,1,0,16.0,9.0,38.0,22.0,5.0,False
1,14.0,2.0,1.0,2.0,32.0,151.27,0.14,7.00,21.11,936.71,...,11,0,1,0,11.0,11.0,18.0,14.0,3.0,False
2,11.0,2.0,1.0,2.0,45.0,197.65,0.11,8.05,22.76,1754.01,...,8,0,1,0,12.0,11.0,28.0,17.0,3.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101760,26.0,1.0,1.0,1.0,83.0,360.17,0.04,22.75,12.56,5893.69,...,20,0,4,0,10.0,8.0,51.0,28.0,1.0,False
101761,10.0,2.0,1.0,2.0,43.0,191.76,0.15,6.46,29.53,1534.04,...,7,0,1,0,11.0,14.0,24.0,19.0,3.0,False
101762,136.0,18.0,18.0,1.0,296.0,1704.57,0.02,44.82,44.08,77011.02,...,102,1,11,0,22.0,31.0,162.0,135.0,35.0,False


In [22]:
# PCA

In [23]:
# data split for baseline model
from sklearn.model_selection import train_test_split

X = train.drop(columns=['defects'])
y = train.defects

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.05, random_state=61)

In [24]:
print(X_train.shape, y_train.shape, X_val.shape, y_val.shape)

(96674, 21) (96674,) (5089, 21) (5089,)


## Baseline Model

In [25]:
model = LGBMClassifier()

model.fit(X_train, y_train)

[LightGBM] [Info] Number of positive: 21922, number of negative: 74752
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.016423 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3581
[LightGBM] [Info] Number of data points in the train set: 96674, number of used features: 21
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.226762 -> initscore=-1.226685
[LightGBM] [Info] Start training from score -1.226685


LGBMClassifier()

In [26]:
print('======Prediction======')
pred_train = model.predict(X_train)
pred_val = model.predict(X_val)

train_score = roc_auc_score(y_train, pred_train)
val_score = roc_auc_score(y_val, pred_val)

print("Train Score : %.4f" % train_score)
print("Test Score : %.4f" % val_score)

======Prediction======
Train Score : 0.6760
Test Score : 0.6630


## Optuna

In [31]:
def optimizer(trial, X, y, K) :
  num_leaves = trial.suggest_categorical('num_leaves', [128, 256, 512])
  max_depth = trial.suggest_int('max_depth', 10, 15)
  learning_rate = trial.suggest_float('learning_rate', 0.01, 0.3)

  model = LGBMClassifier(num_leaves=num_leaves,
                         max_depth=max_depth,
                         learning_rate=learning_rate,
                         random_state=61)

  folds = StratifiedKFold(shuffle=True, random_state=61)
  oof = np.full(len(train), np.nan)
  auc_list = []

  for train_idx, val_idx in folds.split(X, y) :
    X_train = X.iloc[train_idx, :]
    y_train = y.iloc[train_idx]

    X_val = X.iloc[val_idx, :]
    y_val = y.iloc[val_idx]

    model.fit(X_train, y_train)

    try :
      y_val_pred = model.predict_proba(X_val)[:, 1]
    except :
      y_val_pred = model.decision_function(X_val)
    oof[val_idx] = y_val_pred
    auc = roc_auc_score(y_val, y_val_pred)
    auc_list.append(auc)

  return np.mean(auc_list)

In [39]:
K = 5
opt_func = partial(optimizer, X=X_train, y=y_train, K=K)

study = optuna.create_study(direction='maximize')
study.optimize(opt_func, n_trials=50)

[I 2023-10-13 02:57:13,592] A new study created in memory with name: no-name-4c8d3a51-357a-4e9c-8341-52e1719dcad3


[LightGBM] [Info] Number of positive: 17538, number of negative: 59801
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.008360 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3548
[LightGBM] [Info] Number of data points in the train set: 77339, number of used features: 21
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.226768 -> initscore=-1.226652
[LightGBM] [Info] Start training from score -1.226652
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best 

[I 2023-10-13 02:57:28,046] Trial 0 finished with value: 0.7865451937888092 and parameters: {'num_leaves': 256, 'max_depth': 11, 'learning_rate': 0.06492744707306294}. Best is trial 0 with value: 0.7865451937888092.


[LightGBM] [Info] Number of positive: 17538, number of negative: 59801
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.008507 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3548
[LightGBM] [Info] Number of data points in the train set: 77339, number of used features: 21
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.226768 -> initscore=-1.226652
[LightGBM] [Info] Start training from score -1.226652
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best 

[I 2023-10-13 02:57:39,016] Trial 1 finished with value: 0.7735726411750488 and parameters: {'num_leaves': 128, 'max_depth': 14, 'learning_rate': 0.22556677260555544}. Best is trial 0 with value: 0.7865451937888092.


...

[I 2023-10-13 03:08:07,825] Trial 47 finished with value: 0.7905987835514704 and parameters: {'num_leaves': 128, 'max_depth': 15, 'learning_rate': 0.011988946381729365}. Best is trial 35 with value: 0.7909634897366551.


[LightGBM] [Info] Number of positive: 17538, number of negative: 59801
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.008652 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3548
[LightGBM] [Info] Number of data points in the train set: 77339, number of used features: 21
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.226768 -> initscore=-1.226652
[LightGBM] [Info] Start training from score -1.226652
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best 

[I 2023-10-13 03:08:30,428] Trial 48 finished with value: 0.7830087182719592 and parameters: {'num_leaves': 512, 'max_depth': 15, 'learning_rate': 0.04865980000981326}. Best is trial 35 with value: 0.7909634897366551.


[LightGBM] [Info] Number of positive: 17538, number of negative: 59801
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.013229 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3548
[LightGBM] [Info] Number of data points in the train set: 77339, number of used features: 21
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.226768 -> initscore=-1.226652
[LightGBM] [Info] Start training from score -1.226652
[LightGBM] [Info] Number of positive: 17538, number of negative: 59801
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.008340 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3553
[LightGBM] [Info] Number of data points in the train set: 77339, number of used features: 21
[LightGBM] [

[I 2023-10-13 03:08:47,657] Trial 49 finished with value: 0.7882201562981985 and parameters: {'num_leaves': 256, 'max_depth': 14, 'learning_rate': 0.033585051341531516}. Best is trial 35 with value: 0.7909634897366551.


In [40]:
study.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_learning_rate,params_max_depth,params_num_leaves,state
0,0,0.786545,2023-10-13 02:57:13.596193,2023-10-13 02:57:28.046427,0 days 00:00:14.450234,0.064927,11,256,COMPLETE
1,1,0.773573,2023-10-13 02:57:28.048495,2023-10-13 02:57:39.016613,0 days 00:00:10.968118,0.225567,14,128,COMPLETE
2,2,0.776048,2023-10-13 02:57:39.018340,2023-10-13 02:57:52.937972,0 days 00:00:13.919632,0.149970,15,256,COMPLETE
3,3,0.789563,2023-10-13 02:57:52.939702,2023-10-13 02:58:04.694726,0 days 00:00:11.755024,0.053410,13,128,COMPLETE
4,4,0.773056,2023-10-13 02:58:04.702090,2023-10-13 02:58:15.552514,0 days 00:00:10.850424,0.228987,10,256,COMPLETE
5,5,0.790698,2023-10-13 02:58:15.554293,2023-10-13 02:58:29.022694,0 days 00:00:13.468401,0.012735,15,128,COMPLETE
6,6,0.769884,2023-10-13 02:58:29.024446,2023-10-13 02:58:40.564398,0 days 00:00:11.539952,0.258226,10,512,COMPLETE
7,7,0.757078,2023-10-13 02:58:40.566131,2023-10-13 02:58:54.909880,0 days 00:00:14.343749,0.277038,13,512,COMPLETE
8,8,0.775712,2023-10-13 02:58:54.912004,2023-10-13 02:59:09.372518,0 days 00:00:14.460514,0.139144,12,512,COMPLETE
9,9,0.785456,2023-10-13 02:59:09.374431,2023-10-13 02:59:20.546757,0 days 00:00:11.172326,0.102837,13,128,COMPLETE


In [41]:
print('Best Score: %.4f' % study.best_value)
print('Best Params: ', study.best_trial.params)

Best Score: 0.7910
Best Params:  {'num_leaves': 128, 'max_depth': 15, 'learning_rate': 0.021023477843208488}


In [42]:
optuna.visualization.plot_optimization_history(study)

## Test & Submission file

In [54]:
def oof_preds(best_model) :
  folds = StratifiedKFold(shuffle=True, random_state=61)
  final_preds = []
  auc_list = []

  for i, (train_idx, val_idx) in enumerate(folds.split(X, y)) :
    X_train = X.iloc[train_idx, :]
    y_train = y.iloc[train_idx]
    X_val = X.iloc[val_idx, :]
    y_val = y.iloc[val_idx]

    print(f"========== Fold {i+1} ==========")
    best_model.fit(X_train, y_train)
    preds = best_model.predict_proba(X_val)[:, 1]
    test_preds = best_model.predict_proba(test)[:, 1]
    final_preds.append(test_preds)
    auc = roc_auc_score(y_val, preds)

    auc_list.append(auc)

  print(f'AUC :', np.mean(auc))
  return final_preds

In [55]:
best_params = study.best_trial.params

best_model = LGBMClassifier(**best_params,
                            random_state=61)

preds = oof_preds(best_model)
preds = np.mean(preds, axis=0)

preds

========== Fold 1 ==========
[LightGBM] [Info] Number of positive: 18451, number of negative: 62959
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.009400 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3547
[LightGBM] [Info] Number of data points in the train set: 81410, number of used features: 21
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.226643 -> initscore=-1.227365
[LightGBM] [Info] Start training from score -1.227365
========== Fold 2 ==========
[LightGBM] [Info] Number of positive: 18451, number of negative: 62959
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.009073 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3546
[LightGBM] [Info] Number of data points in the 

array([0.22110813, 0.18789681, 0.62623609, ..., 0.16382078, 0.11530813,
       0.74828833])

In [57]:
submission.defects = preds

,defects
id,
101763,0.221108
101764,0.187897
101765,0.626236
101766,0.452574
101767,0.150287
...,...
169600,0.270223
169601,0.128791
169602,0.163821


In [58]:
submission.to_csv('submission_lightGBM.csv')